In [1]:
with open('api.txt','r') as r:
    TEXT = r.read()

In [2]:
from ecommerce.src_base_scraper.scraper_builder import ScraperManagerBuilder
from ecommerce.src_base_scraper.requests_handler import RequestsHandler
from ecommerce.scraper_data.functions import get_pages,get_product,get_product_data

In [3]:
from ecommerce.data_parsing.iterator_parser import EcommerceFuncGenerator

In [4]:
handler = RequestsHandler.create_instance(None)
scrfunc = EcommerceFuncGenerator(requests_handler=handler,
                                 first_url = 'https://fragrancewholesale.co.uk/collections/chatler?page={2}',
                                 max_page = 6
                                 )
func = scrfunc.generate_url_function()
builder = ScraperManagerBuilder(requests_handler = handler,
                      initial_url='https://fragrancewholesale.co.uk/collections/chatler?page=1',
                      initial_node_function = get_pages,
                      node_functions=[get_product],
                      product_functions = get_product_data
                      )

In [5]:
scraper = builder.build()

In [6]:
a = scraper.get_data(parallel_processing_flag=False)

In [7]:
from ecommerce.src_base_scraper.product_info import ProductData

In [8]:
import sqlite3
from typing import Generator

# Your existing code for ProductInfo and ProductData here

def generate_table_sql(product_data: ProductData) -> str:
    """Generate the CREATE TABLE SQL statement based on the keys of product_info."""
    table_columns = ', '.join(f'{key} TEXT' for key in product_data.product_info_keys)
    return f'''
        CREATE TABLE IF NOT EXISTS product_table (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            url TEXT,
            raw_text_response TEXT,
            {table_columns}
        )
    '''

def convert_tag_to_string(tag):
    """Convert Tag object to a suitable string representation."""
    # Adjust this function based on the structure of your Tag object
    return str(tag)

def save_to_database(generator: Generator[ProductData, None, None], database_path: str):
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()

    for product_data in generator:
        # Convert Tag objects to strings or another suitable representation
        product_info_values = [convert_tag_to_string(value) for value in product_data.product_info.values()]

        # Generate CREATE TABLE SQL statement based on the keys of product_info
        create_table_sql = generate_table_sql(product_data)
        cursor.execute(create_table_sql)

        # Insert data into the database
        cursor.execute('''
            INSERT INTO product_table (url, raw_text_response, {}) VALUES (?, ?, {})
        '''.format(', '.join(product_data.product_info_keys), ', '.join(['?'] * len(product_data.product_info_keys))),
        (product_data.url, product_data.raw_text_response, *product_info_values))

    connection.commit()
    connection.close()

# Example usage:
# Assuming you have a generator named product_generator
# and you want to save the data to a SQLite database named 'products.db'


In [9]:
#save_to_database(a,'products2.db')

In [10]:
import aiohttp
import asyncio
import os

async def download_url(session, url, destination_folder):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                # Extract the filename from the URL
                filename = os.path.join(destination_folder, url.split("/")[-1])
                async with aiofiles.open(filename, 'wb') as file:
                    content = await response.read()
                    await file.write(content)
                    print(f"Downloaded: {url}")
            else:
                print(f"Failed to download: {url} (Status Code: {response.status})")
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")

async def download_all_urls(urls, destination_folder):
    async with aiohttp.ClientSession() as session:
        tasks = [download_url(session, url, destination_folder) for url in urls]
        await asyncio.gather(*tasks)

if __name__ == "__main__":
    # Example usage
    urls_generator = (i.product_info.get('image') for i in a)
    destination_folder = "downloads"

    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)
    loop = asyncio.get_event_loop()
    loop.run_until_complete(download_all_urls(urls_generator, destination_folder))


RuntimeError: Cannot run the event loop while another loop is running